<a href="https://colab.research.google.com/github/AnasGamal/big-data-notes-fall-2025/blob/main/001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing tensorflow and keras

import tensorflow as tf
from tensorflow import keras # for 1. Loading

import numpy as np
# for 3. Inspecting
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical, plot_model

In [ ]:
print(tf.__version__)

2.19.0


In [ ]:
# 1. Loading the dataset
cifar10 = keras.datasets.cifar10 #Loading dataset

(xtrain,ytrain), (xtest,ytest) = cifar10.load_data()
print(ytrain.shape)
print(ytest.shape)
print(xtrain.shape)
print(xtest.shape)
print(ytrain)


In [ ]:
# 2. Defining array. Each item represents a value of labels
'''Note, this works because for example, if we define an array and place "airplane" in index 0, we are assuming that
the dataset also places "airplane" in index 0 for its labels
If the dataset "ciraf10" places "airplane" in index 3 for example, the following array won't work as we intend'''

class_names = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
print(class_names)

In [ ]:
# 3. Inspect the data in the array
import random as rd

# we can change this to look at whichever index we want to inspect. I am randomizing this to make it more interesting
# it will change everytime you run this cell
index=rd.randint(0, 100) # change this number
plt.imshow(xtrain[index], cmap=plt.cm.binary) # printing 10th image. You may use cmap='gray'
plt.colorbar() # shows the bar on the right side of the image
plt.grid(True) # will show the grid
plt.show()
print("Class ID: %s and Class name: %s" % (ytrain[index], class_names[ytrain[index][0]]))

In [ ]:
# 4. display the first 25 images from training set

plt.figure(figsize=(10,10))
for i in range(25): # 25 images
  plt.subplot(5,5,i+1) # matrix of 5 X 5 array
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(xtrain[i], cmap=plt.cm.binary) # printing binary/black and white image
    # The CIFAR labels happen to be arrays,
    # which is why you need the extra index
  plt.xlabel("%s %s" % (ytrain[i], class_names[ytrain[i][0]])) # Assigning name to each image
plt.show()

In [ ]:
# Pixel value of the image falls between 0 to 255.
'''This is a process called scaling, it makes training the model faster? Need more info about this'''

xtrain = xtrain/255 # So, we are scale the value between 0 to 1 before by deviding each value by 255
print(xtrain.shape)

xtest = xtest/255 # So, we are scale the value between 0 to 1 before by deviding each value by 255
print(xtest.shape)

In [ ]:
# One hot encoding of the labels.
'''
# convert
# (ytrain)                  (1 corrospends to label)
#                   ["airplane","automobile","bird","cat","deer", "dog"]
# [6]                           [0 0 1 0 0 0]
# [9]                           [0 0 0 0 0 1]
# [9] ------------------>       [0 0 0 0 0 1]
# [.]                              50000x10
# [.]
# 50000x1
#(generally we do one hot encoding of the features in EDA but in this case we are doing it for labels)
'''

# Before one hot encoding
print("ytrain Shape: %s and value: %s" % (ytrain.shape, ytrain))
print("ytest Shape: %s and value: %s" % (ytest.shape, ytest))

ytrain=to_categorical(ytrain)
ytest=to_categorical(ytest)

# After one hot encoding
print("ytrain Shape: %s and value: %s" % (ytrain.shape, ytrain[0]))
print("ytest Shape: %s and value: %s" % (ytest.shape, ytest[1]))

In [ ]:
# Modelling - Model on CNN

from tensorflow.keras import models, layers
from keras.layers import Conv2D, MaxPooling2D, Flatten , Dense, Activation,Dropout

# create a sequential model i.e. empty neural network which has no layers in it.
model=models.Sequential()

#==================== Feature Detection / extraction Block ====================#

'''The first layer is the first add() invocation
[32,32, 3] -> [64,3] -> [128,1]
Need to figure out how to determine layer dimensions
'''
# Add first convolutional block - To deal with images we use Conv2D and for colour images and shape use Conv3D
#model.add(layers.Conv2D(filters=6, kernal_size(3,3), input_shape=(32,32,1), activation='relu'))
# in the first block we need to mention input_shape
model.add(layers.Conv2D(64,(3,3),input_shape=(32,32,3),activation='relu'))
model.add(layers.Conv2D(64,(3,3),input_shape=(32,32,3),activation='relu'))
# Add the max pooling layer
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Add Second convolutional block
#model.add(layers.Conv2D(filters=6, kernal_size(3,3), activation='relu'))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
# Add the max pooling layer
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Add Third convolutional block
#model.add(layers.Conv2D(filters=6, kernal_size(3,3), activation='relu'))
model.add(layers.Conv2D(256,(3,3),activation='relu'))
# model.add(layers.Conv2D(256,(3,3),activation='relu'))
# Add the max pooling layer
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


#==================== Transition Block (from feature detection to classification) ====================#

# Add Flatten layer. Flatten simply converts matrics to array
model.add(layers.Flatten(input_shape=(32,32))) # this will flatten the image and after this Classification happens

#==================== Classification Block ====================#

# Classification segment - fully connected network
# The Dense layer does classification and is deep neural network. Dense layer always accept the array.

# Where did 128 come from?
model.add(layers.Dense(128, activation='relu')) # as C5 layer in above image.
# Where did 100 come from?
model.add(layers.Dense(100, activation='relu')) # as C5 layer in above image.
# Where did 80 come form?
model.add(layers.Dense(80, activation='relu')) # as C5 layer in above image.
# model.add(layers.Dense(60, activation='relu')) # as C5 layer in above image
# model.add(layers.Dense(40, activation='relu')) # as C5 layer in above image
# this 120 is hyper parameter whcih is number of neuron
#model.add(layers.Dense(84, activation='relu'))# as F6 layer in aboave image

# Add the output layer
model.add(layers.Dense(10, activation='softmax')) # as Output layer in above image. The output layer normally have softmax activation

# Ploting the Model
plot_model(model)